<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=527d274368755906c86eea064e54480cc603587cd1819a8efcb9d3edbd7f6374
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-01 09:36:29
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 14


-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.43 C
-------------------
Today PnL: 5.58 K (0.04%)
Current PnL: -25.90 L (-16.52%)
CY Booked + Current PnL: -11.61 L (-7.4%)
-------------------
Total profit:  1.61 L
Total loss:  -27.51 L
-------------------
Total Booked + Current PnL: 15.37 L (11.84%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.98%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 91.01 L (63.57%)
Deployed:  1.30 C
Current:  1.43 C
CAGR/XIRR %: 5.4%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,NESTLEIND,1377.00,0.48,14.10,6.40,21.40,19382.0,37417.0,302843.0,5.82,68.0,X-LC,5.15,12.0,1.93,2.12,21.32,XY25,NTT,FMCG
78,TTKPRESTIG,770.00,0.25,-19.60,24.49,0.09,19843.0,-19753.0,81024.0,88.05,40.0,M-SC,3.36,253.0,-1.00,0.57,6.45,OX40N,NTT,DURABLES
50,MASFIN,398.61,-0.08,-1.00,23.28,22.05,22583.0,-975.0,97005.0,-14.63,56.0,H-SC,6.31,164.0,-0.04,0.68,41.00,XR,ATH,FINANCE
83,VOLTAS,1530.00,0.86,7.40,11.44,19.69,23559.0,14193.0,205935.0,-3.01,50.0,X-MC,2.04,78.0,0.60,1.44,14.70,XY25,NTT,AC
40,INDIGOPNTS,1408.00,3.21,-16.14,19.27,0.02,28208.0,-28177.0,146382.0,123.21,49.0,M-SC,7.04,234.0,-1.00,1.02,27.33,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDIGOPNTS,1408.0,3.21,-16.14,19.27,0.02,28208.0,-28177.0,146382.0,123.21,49.0,M-SC,7.04,234.0,-1.00,1.02,27.33,OX40N,NTT,PAINTS
58,RECLTD,446.0,1.54,-1.92,23.10,20.74,46030.0,-3905.0,199265.0,28.81,61.0,M-MC,8.32,189.0,-0.08,1.39,8.49,XY25,NTT,FINANCE
60,RELIANCE,1952.0,1.18,2.00,22.84,25.30,46818.0,4018.0,204981.0,-1.18,65.0,X-LC,9.17,17.0,0.09,1.43,37.30,XY25,BTT,REFINERIES
69,STARHEALTH,761.0,1.10,-10.99,65.43,47.26,163732.0,-30883.0,250240.0,21.44,46.0,H-SC,9.32,171.0,-0.19,1.75,34.07,XY24,NTT,INSURANCE
0,5PAISA,593.0,1.02,-35.79,75.83,12.90,106386.0,-78200.0,140296.0,106.73,59.0,H-SC,12.09,161.0,-0.74,0.98,16.66,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,-3.61,-4.79,16.36,10.78,37431.0,-11521.0,228797.0,-43.20,30.0,X-LC,2.17,5.0,-0.31,1.60,0.19,X40,NTT,FMCG
29,GREENPANEL,537.00,-1.11,-39.60,136.97,43.14,151470.0,-72492.0,110586.0,147.42,37.0,M-SC,12.06,240.0,-0.48,0.77,4.02,XY24,NTT,MISC
53,PAGEIND,51605.08,-0.96,-11.37,44.55,28.11,79522.0,-22910.0,178500.0,-35.73,35.0,X-MC,15.32,55.0,-0.29,1.25,0.38,X40,ATH,APPARELS
30,HAPPSTMNDS,1480.71,-0.81,-42.99,224.25,84.85,163846.0,-55101.0,73064.0,-32.30,32.0,H-SC,14.21,147.0,-0.34,0.51,0.55,AR,ATH,IT
28,GLAXO,3466.20,-0.78,-1.48,41.24,39.15,80969.0,-2948.0,196336.0,-20.92,41.0,X-MC,10.85,60.0,-0.04,1.37,27.09,X40,ATH,PHARMA


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.08,0.05,81.58,81.67,76571.0,50.0,93860.0,8455.56,67.0,L-SC,17.08,271.0,0.0,0.66,81.76,XR,NTT,CERAMICS


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,FINCABLES,1641.55,0.09,-4.96,118.77,107.91,166652.0,-7329.0,140315.0,-22.58,46.0,M-SC,11.43,216.0,-0.04,0.98,4.62,OX40N,ATH,CABLES
40,INDIGOPNTS,1408.00,3.21,-16.14,19.27,0.02,28208.0,-28177.0,146382.0,123.21,49.0,M-SC,7.04,234.0,-1.00,1.02,27.33,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.25,-19.60,24.49,0.09,19843.0,-19753.0,81024.0,88.05,40.0,M-SC,3.36,253.0,-1.00,0.57,6.45,OX40N,NTT,DURABLES
48,KANSAINER,340.00,-0.31,-21.66,44.85,13.47,94745.0,-58419.0,211248.0,-67.73,53.0,H-SC,6.40,158.0,-0.62,1.48,7.41,XY24,NTT,PAINTS
67,SIS,528.00,-0.72,-23.98,59.52,21.26,50238.0,-26627.0,84405.0,1991.71,49.0,H-SC,4.73,166.0,-0.53,0.59,14.04,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.08,0.05,81.58,81.67,76571.0,50.0,93860.0,8455.56,67.0,L-SC,17.08,271.0,0.00,0.66,81.76,XR,NTT,CERAMICS
85,WIPRO,420.0,0.92,8.98,58.07,72.27,107233.0,15214.0,184662.0,-6.53,60.0,M-LC,5.41,99.0,0.14,1.29,15.71,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.92,8.98,58.07,72.27,107233.0,15214.0,184662.0,-6.53,60.0,M-LC,5.41,99.0,0.14,1.29,15.71,XR,NTT,IT
6,ASIANTILES,137.00,0.08,0.05,81.58,81.67,76571.0,50.0,93860.0,8455.56,67.0,L-SC,17.08,271.0,0.00,0.66,81.76,XR,NTT,CERAMICS
39,INDIAMART,4810.62,-0.17,-4.61,116.71,106.72,137308.0,-5687.0,117649.0,-52.73,42.0,H-SC,6.08,139.0,-0.04,0.82,17.69,AR,ATH,MISC
26,FINCABLES,1641.55,0.09,-4.96,118.77,107.91,166652.0,-7329.0,140315.0,-22.58,46.0,M-SC,11.43,216.0,-0.04,0.98,4.62,OX40N,ATH,CABLES
50,MASFIN,398.61,-0.08,-1.00,23.28,22.05,22583.0,-975.0,97005.0,-14.63,56.0,H-SC,6.31,164.0,-0.04,0.68,41.00,XR,ATH,FINANCE


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HONAUT,58357.33,0.29,-19.20,77.38,43.32,101832.0,-31272.0,131600.0,-30.52,29.0,X-SC,7.52,90.0,-0.31,0.92,1.04,X40N,ATH,ELECTRICAL
44,ITC,452.00,-3.61,-4.79,16.36,10.78,37431.0,-11521.0,228797.0,-43.20,30.0,X-LC,2.17,5.0,-0.31,1.60,0.19,X40,NTT,FMCG
23,DIXON,18931.72,0.15,-19.29,56.20,26.07,88549.0,-37661.0,157560.0,-60.75,31.0,X-MC,16.56,56.0,-0.43,1.10,3.00,X40N,ATH,IT
9,BAJAJHFL,181.50,0.43,-18.85,91.58,55.47,180814.0,-45848.0,197438.0,-28.03,32.0,X-MC,16.53,64.0,-0.25,1.38,1.16,X40N,ATH,FINANCE
53,PAGEIND,51605.08,-0.96,-11.37,44.55,28.11,79522.0,-22910.0,178500.0,-35.73,35.0,X-MC,15.32,55.0,-0.29,1.25,0.38,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.5,0.34,-17.42,51.98,25.51,79885.0,-32410.0,153685.0,-5.62,42.0,H-LC,0.90,49.0,-0.41,1.07,13.25,AR,ATH,ELECTRICAL
79,UNITDSPR,1644.0,-0.17,5.88,14.06,20.77,34855.0,13760.0,247904.0,-3.38,54.0,X-MC,1.59,62.0,0.39,1.73,13.20,X40N,NTT,BREWERIES
13,BERGEPAINT,680.0,-0.62,-5.00,27.52,21.14,59434.0,-11373.0,215966.0,-11.79,43.0,X-MC,1.82,74.0,-0.19,1.51,19.93,XY24,NTT,PAINTS
21,DABUR,735.0,-0.27,-1.80,46.34,43.70,112880.0,-4477.0,243591.0,-9.97,53.0,X-MC,1.96,72.0,-0.04,1.70,12.97,XY24,BTT,FMCG
83,VOLTAS,1530.0,0.86,7.40,11.44,19.69,23559.0,14193.0,205935.0,-3.01,50.0,X-MC,2.04,78.0,0.60,1.44,14.70,XY25,NTT,AC


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,ITC,452.00,-3.61,-4.79,16.36,10.78,37431.0,-11521.0,228797.0,-43.20,30.0,X-LC,2.17,5.0,-0.31,1.60,0.19,X40,NTT,FMCG
53,PAGEIND,51605.08,-0.96,-11.37,44.55,28.11,79522.0,-22910.0,178500.0,-35.73,35.0,X-MC,15.32,55.0,-0.29,1.25,0.38,X40,ATH,APPARELS
12,BATAINDIA,2096.00,0.14,-39.08,121.95,35.20,95586.0,-50289.0,78381.0,2.82,36.0,X-SC,17.34,92.0,-0.53,0.55,0.67,X40,NTT,FOOTWEAR
3,ACC,3906.00,0.08,-26.91,124.60,64.15,216692.0,-64041.0,173910.0,-57.19,37.0,X-SC,5.80,82.0,-0.30,1.21,0.81,XY24,BTT,CEMENT
35,HONAUT,58357.33,0.29,-19.20,77.38,43.32,101832.0,-31272.0,131600.0,-30.52,29.0,X-SC,7.52,90.0,-0.31,0.92,1.04,X40N,ATH,ELECTRICAL


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,QUESS,424.00,0.68,-30.27,104.85,42.84,47527.0,-19677.0,45329.0,-54.27,42.0,X-SC,31.15,83.0,-0.41,0.32,2.55,XY24,NTT,MISC
59,RELAXO,1176.00,0.09,-47.58,190.55,52.31,144995.0,-69067.0,76093.0,-44.15,46.0,X-SC,7.40,91.0,-0.48,0.53,1.85,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,0.14,-39.08,121.95,35.20,95586.0,-50289.0,78381.0,2.82,36.0,X-SC,17.34,92.0,-0.53,0.55,0.67,X40,NTT,FOOTWEAR
51,MEDANTA,1486.00,0.86,-0.09,24.19,24.08,30683.0,-110.0,126840.0,-8.22,53.0,X-SC,7.61,89.0,-0.00,0.89,19.51,XY24,NTT,HEALTHCARE
35,HONAUT,58357.33,0.29,-19.20,77.38,43.32,101832.0,-31272.0,131600.0,-30.52,29.0,X-SC,7.52,90.0,-0.31,0.92,1.04,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.05,-11.88,36.99,20.71,123284.0,-44948.0,333289.0,-23.95,47.0,X-LC,2.90,1.0,-0.36,2.33,11.36,X40,ATH,IT
42,INFY,1972.00,0.23,10.22,21.80,34.24,75181.0,31980.0,344868.0,-14.21,53.0,X-LC,7.38,2.0,0.43,2.41,19.35,X40,NTT,IT
76,TMPV,600.00,-0.11,-24.79,63.51,22.98,107902.0,-56000.0,169898.0,-21.86,58.0,X-LC,5.69,3.0,-0.52,1.19,6.86,XY24,NTT,AUTO
82,VBL,671.64,0.28,-0.78,36.73,35.67,115107.0,-2456.0,313386.0,-12.64,63.0,X-LC,2.80,4.0,-0.02,2.19,12.97,X40N,ATH,FMCG
44,ITC,452.00,-3.61,-4.79,16.36,10.78,37431.0,-11521.0,228797.0,-43.20,30.0,X-LC,2.17,5.0,-0.31,1.60,0.19,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INDUSINDBK,1800.00,0.14,-36.11,108.00,32.89,53274.0,-27878.0,49328.0,-709.47,61.0,L-MC,6.57,259.0,-0.52,0.34,35.84,XR,NTT,BANKS
6,ASIANTILES,137.00,0.08,0.05,81.58,81.67,76571.0,50.0,93860.0,8455.56,67.0,L-SC,17.08,271.0,0.00,0.66,81.76,XR,NTT,CERAMICS
50,MASFIN,398.61,-0.08,-1.00,23.28,22.05,22583.0,-975.0,97005.0,-14.63,56.0,H-SC,6.31,164.0,-0.04,0.68,41.00,XR,ATH,FINANCE
14,BSOFT,831.70,-0.72,-22.01,93.33,50.79,102384.0,-30952.0,109701.0,0.12,53.0,H-SC,10.80,151.0,-0.30,0.77,27.73,XR,ATH,IT
70,SURYODAY,216.00,0.30,-19.53,52.90,23.03,76226.0,-34976.0,144095.0,57.89,54.0,H-SC,6.56,167.0,-0.46,1.01,42.97,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.0,0.08,0.05,81.58,81.67,76571.0,50.0,93860.0,8455.56,67.0,L-SC,17.08,271.0,0.00,0.66,81.76,XR,NTT,CERAMICS
14,BSOFT,831.7,-0.72,-22.01,93.33,50.79,102384.0,-30952.0,109701.0,0.12,53.0,H-SC,10.80,151.0,-0.30,0.77,27.73,XR,ATH,IT
75,TITAGARH,1548.0,-0.13,-16.13,73.78,45.74,149849.0,-39075.0,203102.0,11.66,64.0,H-SC,4.38,174.0,-0.26,1.42,32.10,XY24,NTT,ENGINEERING
37,ICICIPRULI,790.0,0.45,11.08,17.69,30.73,38711.0,21823.0,218828.0,-14.84,71.0,X-MC,4.76,75.0,0.56,1.53,25.26,X40,ATH,INSURANCE
85,WIPRO,420.0,0.92,8.98,58.07,72.27,107233.0,15214.0,184662.0,-6.53,60.0,M-LC,5.41,99.0,0.14,1.29,15.71,XR,NTT,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.99
1,25,44.06
2,50,75.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.37
MC    30.19
LC    26.46
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.50
X40      23.82
X40N     12.63
XY25     10.48
XR        9.64
AR        8.59
OX40N     7.38
SR        0.98
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    23.77
X-LC    23.07
H-SC    22.72
M-SC    11.31
X-SC     7.86
H-MC     4.69
L-SC     1.48
M-MC     1.39
M-LC     1.29
H-LC     1.07
L-LC     1.03
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.04,-6.94,41.33
IT,12.51,-17.51,76.60
FINANCE,10.90,-14.42,62.44
MISC,6.98,-30.05,83.25
ELECTRICAL,5.82,-12.61,53.86
PAINTS,5.71,-13.02,29.80
INSURANCE,4.73,-0.30,34.62
PHARMA,3.98,-2.93,35.34
AUTO,2.82,-29.97,75.03


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3131212.0,21
AR,1339619.0,10
XR,1284139.0,13
X40,1061615.0,15
X40N,877543.0,9
OX40N,735521.0,10
XY25,391403.0,7
SR,280069.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3545181.0,24
M-SC,1439199.0,15
X-MC,1425051.0,16
X-LC,923687.0,13
X-SC,793551.0,8
H-MC,406999.0,3
L-SC,249713.0,3
M-LC,107233.0,1
H-LC,79885.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1235388.0      6
           AR         916383.0      5
M-SC       XY24       815652.0      6
H-SC       XR         784438.0      7
X-MC       X40        482421.0      7
           XY24       407844.0      3
X-LC       X40        397110.0      6
X-MC       X40N       348235.0      4
H-SC       OX40N      328903.0      4
M-SC       OX40N      317820.0      5
X-SC       X40N       316565.0      3
           XY24       294902.0      3
H-SC       SR         280069.0      2
H-MC       AR         215903.0      2
X-LC       X40N       212743.0      2
H-MC       XY24       191096.0      1
X-MC       XY25       186551.0      2
X-LC       XY24       186330.0      2
X-SC       X40        182084.0      2
M-SC       XR         178279.0      2
L-SC       XR         160915.0      2
X-LC       XY25       127504.0      3
M-SC       AR         127448.0      2
M-LC       XR         107233.0      1
L-SC       OX40N       88798.0      1
H-LC       AR          79885.0      1
L-MC       XR          53274.0      1
M-MC       XY25        46030.0      1
L-LC       XY25        31318.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 41.0 seconds
